## Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Dependencies and Setup
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
# Go to existing database with automap_base
from sqlalchemy.ext.automap import automap_base
# Work through mapper to use python code
from sqlalchemy.orm import Session, relationship
# Inspect with python
from sqlalchemy import create_engine, inspect
from config import APIKEY

In [2]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [3]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

-------------

## All_Data import
Will only need to run once since its 20 years of data and does not change as much.

In [4]:
with open('../../datasets/_2020_yil_all_per_save.json') as json_file:
    data_all = json.load(json_file)

In [5]:
import locale

In [6]:
all_data_logs = pd.read_json(data_all).copy()
all_data_logs

new_values_once = []
for value in all_data_logs['INFLATION']:
    sala = value
    locale.setlocale(locale.LC_ALL, '')
    val = locale.currency(sala, symbol=False, grouping=True)
    new_values_once.append(val)
all_data_logs['INFLATION'] = new_values_once

new_values_SAL_PRED_once = []
for value in all_data_logs['SAL_PRED']:
    sala = value
    locale.setlocale(locale.LC_ALL, '')
    val = locale.currency(sala, symbol=False, grouping=True)
    new_values_SAL_PRED_once.append(val)
all_data_logs['SAL_PRED'] = new_values_SAL_PRED_once


In [7]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [8]:
# Use this to clear out the db
# ----------------------------------
# # Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

In [9]:
## Class base template to upload to sqlite
class TopAllYil5(Base):
    __tablename__ = 'topAllYil5'
 
    id = Column(Integer, primary_key=True)
    POSITION = Column(String(3))
    PLAYER_NAME = Column(String(70))
    PHOTO = Column(String())
    YEAR = Column(Float)
    YIL = Column(Float)
    ROUND_NUMBER = Column(Float)
    OVERALL_PICK = Column(Float)
    MIN_PRED = Column(Float)
    PER = Column(Float)
    INJURY_PY = Column(Float)
    INFLATION = Column(String(70))
    SAL_PRED = Column(String(70))
    SAL_RISIDUAL = Column(Float)


In [10]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

In [11]:
# Function: Add Weather Series to sql
def addToSQL(all_logs):
    x=0
    while x <= (len(all_logs) - 1):
#         try: 
        TopAll_Yil5 = TopAllYil5(
            POSITION = all_logs["POSITION"][x],
            PLAYER_NAME = all_logs["PLAYER_NAME"][x],
            PHOTO = all_logs['PHOTO'][x],
            YEAR = all_logs["YEAR"][x],
            YIL = all_logs["YIL"][x],
            ROUND_NUMBER = all_logs["ROUND_NUMBER"][x],
            OVERALL_PICK = all_logs["OVERALL_PICK"][x],
            MIN_PRED = all_logs["MIN_PRED"][x],
            PER = all_logs["PER"][x],
            INJURY_PY = all_logs["INJURY_PY"][x],
            INFLATION = all_logs["INFLATION"][x],
            SAL_PRED = all_logs["SAL_PRED"][x],
            SAL_RISIDUAL = all_logs["SAL_RISIDUAL"][x],
                )
        x+=1
#         except OperationalError as e:
#             pass
            
        # Add Records to the Appropriate DB
        # ----------------------------------
        # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
        session.add(TopAll_Yil5)
        # session.add(earthquake_data)
        session.commit()
    return 'Complete: Uploaded to SQLite DB'

In [12]:
addToSQL(all_data_logs)

'Complete: Uploaded to SQLite DB'

## Open Saved JSON

In [13]:
with open('../../datasets/_2020_yil_3_per_only_save.json') as json_file:
    data = json.load(json_file)

In [14]:
import locale

In [15]:
all_logs = pd.read_json(data).copy()
all_logs

new_values = []
for value in all_logs['INFLATION']:
    sala = value
    locale.setlocale(locale.LC_ALL, '')
    val = locale.currency(sala, symbol=False, grouping=True)
    new_values.append(val)
all_logs['INFLATION'] = new_values

new_values_SAL_PRED = []
for value in all_logs['SAL_PRED']:
    sala = value
    locale.setlocale(locale.LC_ALL, '')
    val = locale.currency(sala, symbol=False, grouping=True)
    new_values_SAL_PRED.append(val)
all_logs['SAL_PRED'] = new_values_SAL_PRED



In [16]:
all_logs

,PLAYER_ID,POSITION,PLAYER_NAME,PHOTO,SEASON_ID,YEAR,PLAYER_AGE,YIL,ROUND_NUMBER,OVERALL_PICK,...,PTS,PER,ALLSTAR,INJURY_PY,MIN,MIN_PRED,RISIDUAL,INFLATION,SAL_PRED,SAL_RISIDUAL
0,1629029,SG,Luka Doncic,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,21,2,1,3,...,1759,1997.61,0,0,2047,2666.25,1.303,"7,683,360.00","8,993,686.00",1.17
1,1628369,PF,Jayson Tatum,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,22,3,1,3,...,1547,1469.99,0,1,2265,2676.47,1.182,"7,830,000.00","8,241,704.00",1.05
2,1629027,PG,Trae Young,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,21,2,1,5,...,1778,1736.48,0,2,2120,2471.17,1.166,"6,273,000.00","8,143,110.00",1.30
3,1628378,SG,Donovan Mitchell,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,23,3,1,13,...,1655,1304.48,0,0,2364,2697.30,1.141,"3,635,760.00","7,309,705.00",2.01
4,1628389,PF,Bam Adebayo,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,22,3,1,14,...,1146,1868.04,0,0,2417,2474.03,1.024,"3,454,080.00","7,209,036.00",2.09
5,1628368,PG,De'Aaron Fox,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,22,3,1,5,...,1077,1125.65,0,0,1634,1893.99,1.159,"6,392,760.00","6,329,290.00",0.99
6,1628366,PG,Lonzo Ball,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,22,3,1,2,...,745,997.35,0,0,2025,2060.38,1.017,"8,719,320.00","6,284,409.00",0.72
7,1627732,PG,Ben Simmons,https://s3-us-west-2.amazonaws.com/static.fant...,2019-20,2020,23,3,1,1,...,937,1494.57,0,0,2017,2138.20,1.060,"8,113,930.00","6,086,913.00",0.75
8,1628381,PF,John Collins,./static/images/stock-headshot.png,2019-20,2020,22,3,1,19,...,885,1021.27,0,0,1363,1443.69,1.059,"2,686,560.00","5,700,371.00",2.12
9,1628984,PG,Devonte' Graham,./static/images/stock-headshot.png,2019-20,2020,25,2,2,34,...,1145,1097.80,0,0,2211,2248.52,1.017,"1,416,852.00","5,598,380.00",3.95


In [17]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [18]:
# Use this to clear out the db
# ----------------------------------
# # Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

In [19]:
## Class base template to upload to sqlite
class Top2020Yil5(Base):
    __tablename__ = 'top2020Yil5'
 
    id = Column(Integer, primary_key=True)
    POSITION = Column(String(3))
    PLAYER_NAME = Column(String(70))
    PHOTO = Column(String())
    YEAR = Column(Float)
    YIL = Column(Float)
    ROUND_NUMBER = Column(Float)
    OVERALL_PICK = Column(Float)
    MIN_PRED = Column(Float)
    PER = Column(Float)
    INJURY_PY = Column(Float)
    INFLATION = Column(String(70))
    SAL_PRED = Column(String(70))
    SAL_RISIDUAL = Column(Float)


In [20]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

In [21]:
# session.add(top_2020_yil_5)

In [22]:
# session.commit()

In [23]:
# # Create Database Connection
# # ----------------------------------
# # Creates a connection to our DB
# # Engine opens the door. Conn is the walk through sign
# engine = create_engine("sqlite:///db.sqlite")
# conn = engine.connect()
# # Create a "Metadata" Layer That Abstracts our SQL Database
# # ----------------------------------
# # Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [24]:
# Function: Add Weather Series to sql
def addToSQL(all_logs):
    x=0
    while x <= (len(all_logs) - 1):
        try: 
            Top2020_Yil5 = Top2020Yil5(
                POSITION = all_logs["POSITION"][x],
                PLAYER_NAME = all_logs["PLAYER_NAME"][x],
                PHOTO = all_logs['PHOTO'][x],
                YEAR = all_logs["YEAR"][x],
                YIL = all_logs["YIL"][x],
                ROUND_NUMBER = all_logs["ROUND_NUMBER"][x],
                OVERALL_PICK = all_logs["OVERALL_PICK"][x],
                MIN_PRED = all_logs["MIN_PRED"][x],
                PER = all_logs["PER"][x],
                INJURY_PY = all_logs["INJURY_PY"][x],
                INFLATION = all_logs["INFLATION"][x],
                SAL_PRED = all_logs["SAL_PRED"][x],
                SAL_RISIDUAL = all_logs["SAL_RISIDUAL"][x],
                
                )
            x+=1
        except OperationalError:
            pass
            
        # Add Records to the Appropriate DB
        # ----------------------------------
        # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
        session.add(Top2020_Yil5)
        # session.add(earthquake_data)
        session.commit()
    return 'Complete: Uploaded to SQLite DB'

In [25]:
addToSQL(all_logs)

'Complete: Uploaded to SQLite DB'

In [26]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
# Go to existing database with automap_base
from sqlalchemy.ext.automap import automap_base
# Work through mapper to use python code
from sqlalchemy.orm import Session
# Inspect with python
from sqlalchemy import create_engine, inspect
from sqlalchemy import desc


In [27]:
Base = automap_base()
Base.prepare(engine, reflect=True)
# Check db table names
Base.classes.keys()
data_from_db = Base.classes.top2020Yil5
data_container = session.query(data_from_db).limit(10)
analysis_container = []


In [28]:
for data in data_container:
    # get specific data from db
    container = {
        'POSITION': data.POSITION,
        'PLAYER_NAME': data.PLAYER_NAME,
        'PHOTO': data.PHOTO,
        'YEAR': data.YEAR,
        'YIL': data.YIL,
        'ROUND_NUMBER': data.ROUND_NUMBER,
        'OVERALL_PICK': data.OVERALL_PICK,
        'MIN_PRED': data.MIN_PRED,
        'PER': data.PER,
        'INJURY_PY': data.INJURY_PY,
        'INFLATION': data.INFLATION,
        'SAL_PRED': data.SAL_PRED,
        'SAL_RISIDUAL': data.SAL_RISIDUAL,
        
#         "date": data.date,
#         "magnitude": data.magnitude,
#         "maxtemp": data.maxtemp,
#         "mintemp": data.mintemp,
# #         "avgtemp": data.avgtemp,
#         "lat": data.lat,
        }
    analysis_container.append(container)

# Create df for parsing    
temp_df = pd.DataFrame(analysis_container)

In [29]:
temp_df

,POSITION,PLAYER_NAME,PHOTO,YEAR,YIL,ROUND_NUMBER,OVERALL_PICK,MIN_PRED,PER,INJURY_PY,INFLATION,SAL_PRED,SAL_RISIDUAL
0,SG,Luka Doncic,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,2.0,1.0,3.0,2666.25,1997.61,0.0,"7,683,360.00","8,993,686.00",1.17
1,PF,Jayson Tatum,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,3.0,2676.47,1469.99,1.0,"7,830,000.00","8,241,704.00",1.05
2,PG,Trae Young,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,2.0,1.0,5.0,2471.17,1736.48,2.0,"6,273,000.00","8,143,110.00",1.30
3,SG,Donovan Mitchell,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,13.0,2697.30,1304.48,0.0,"3,635,760.00","7,309,705.00",2.01
4,PF,Bam Adebayo,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,14.0,2474.03,1868.04,0.0,"3,454,080.00","7,209,036.00",2.09
5,PG,De'Aaron Fox,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,5.0,1893.99,1125.65,0.0,"6,392,760.00","6,329,290.00",0.99
6,PG,Lonzo Ball,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,2.0,2060.38,997.35,0.0,"8,719,320.00","6,284,409.00",0.72
7,PG,Ben Simmons,https://s3-us-west-2.amazonaws.com/static.fant...,2020.0,3.0,1.0,1.0,2138.20,1494.57,0.0,"8,113,930.00","6,086,913.00",0.75
8,PF,John Collins,./static/images/stock-headshot.png,2020.0,3.0,1.0,19.0,1443.69,1021.27,0.0,"2,686,560.00","5,700,371.00",2.12
9,PG,Devonte' Graham,./static/images/stock-headshot.png,2020.0,2.0,2.0,34.0,2248.52,1097.80,0.0,"1,416,852.00","5,598,380.00",3.95


------------

## UPDATE_DATA

In [30]:
def top2020_yil_5():
    # Sets an object to utilize the default declarative base in SQL Alchemy
    Base = declarative_base()
    Base_all = declarative_base()
    ## Class base template to upload to sqlite
    class Top2020Yil5(Base):
        __tablename__ = 'top2020Yil5'
 
        id = Column(Integer, primary_key=True)
        POSITION = Column(String(3))
        PLAYER_NAME = Column(String(70))
        PHOTO = Column(String())
        YEAR = Column(Float)
        YIL = Column(Float)
        ROUND_NUMBER = Column(Float)
        OVERALL_PICK = Column(Float)
        MIN_PRED = Column(Float)
        PER = Column(Float)
        INJURY_PY = Column(Float)
        INFLATION = Column(String(170))
        SAL_PRED = Column(String(170))
        SAL_RISIDUAL = Column(Float)

        
    class TopAllYil5(Base_all):
        __tablename__ = 'topAllYil5'

        id = Column(Integer, primary_key=True)
        POSITION = Column(String(3))
        PLAYER_NAME = Column(String(70))
        PHOTO = Column(String())
        YEAR = Column(Float)
        YIL = Column(Float)
        ROUND_NUMBER = Column(Float)
        OVERALL_PICK = Column(Float)
        MIN_PRED = Column(Float)
        PER = Column(Float)
        INJURY_PY = Column(Float)
        INFLATION = Column(String(70))
        SAL_PRED = Column(String(70))
        SAL_RISIDUAL = Column(Float)

    # Create Database Connection
    # ----------------------------------
    # Creates a connection to our DB
    # Engine opens the door. Conn is the walk through sign
    engine = create_engine("sqlite:///db.sqlite")
    conn = engine.connect()
    # Create a "Metadata" Layer That Abstracts our SQL Database
    # ----------------------------------
    # Create (if not already in existence) the tables associated with our classes.
    Base.metadata.create_all(engine)
    Base_all.metadata.create_all(engine)
    # Create a Session Object to Connect to DB
    # ----------------------------------
    session = Session(bind=engine)
    both_sets = {}
    def parseDataAndFormat():
        Base = automap_base()
        Base_all = automap_base()
        
        Base.prepare(engine, reflect=True)
        Base_all.prepare(engine, reflect=True)
        
        # Check db table names
        Base.classes.keys()
        data_from_db = Base.classes.top2020Yil5
        all_stats_db = Base_all.classes.topAllYil5
        
        data_container = session.query(data_from_db).limit(10)
        data_container_all = session.query(all_stats_db).limit(10)
        
        
        analysis_container = []
        analysis_container_all = []
        
        for data in data_container:
            # get specific data from db
            container = {
                'POSITION': data.POSITION,
                'PLAYER_NAME': data.PLAYER_NAME,
                'PHOTO': data.PHOTO,
                'YEAR': data.YEAR,
                'YIL': data.YIL,
                'ROUND_NUMBER': data.ROUND_NUMBER,
                'OVERALL_PICK': data.OVERALL_PICK,
                'MIN_PRED': data.MIN_PRED,
                'PER': data.PER,
                'INJURY_PY': data.INJURY_PY,
                'INFLATION': data.INFLATION,
                'SAL_PRED': data.SAL_PRED,
                'SAL_RISIDUAL': data.SAL_RISIDUAL,
                }
            analysis_container.append(container)
            
        for data in data_container_all:
            # get specific data from db
            container2 = {
                'POSITION': data.POSITION,
                'PLAYER_NAME': data.PLAYER_NAME,
                'PHOTO': data.PHOTO,
                'YEAR': data.YEAR,
                'YIL': data.YIL,
                'ROUND_NUMBER': data.ROUND_NUMBER,
                'OVERALL_PICK': data.OVERALL_PICK,
                'MIN_PRED': data.MIN_PRED,
                'PER': data.PER,
                'INJURY_PY': data.INJURY_PY,
                'INFLATION': data.INFLATION,
                'SAL_PRED': data.SAL_PRED,
                'SAL_RISIDUAL': data.SAL_RISIDUAL,
                }
            analysis_container_all.append(container2)
            
        both_sets = {
            'current': analysis_container,
            'historic': analysis_container_all
            }
        return both_sets

    analysis_container = parseDataAndFormat()
    # Return results
    return analysis_container

In [31]:
test = top2020_yil_5()
test['historic']

[{'POSITION': 'SF',
  'PLAYER_NAME': 'LeBron James',
  'PHOTO': 'https://s3-us-west-2.amazonaws.com/static.fantasydata.com/headshots/nba/low-res/20000571.png',
  'YEAR': 2006.0,
  'YIL': 3.0,
  'ROUND_NUMBER': 1.0,
  'OVERALL_PICK': 1.0,
  'MIN_PRED': 3555.26,
  'PER': 2581.49,
  'INJURY_PY': 0.0,
  'INFLATION': '5,756,777.00',
  'SAL_PRED': '9,793,533.00',
  'SAL_RISIDUAL': 1.7000000000000002},
 {'POSITION': 'SF',
  'PLAYER_NAME': 'Kevin Durant',
  'PHOTO': 'https://s3-us-west-2.amazonaws.com/static.fantasydata.com/headshots/nba/low-res/20000884.png',
  'YEAR': 2010.0,
  'YIL': 3.0,
  'ROUND_NUMBER': 1.0,
  'OVERALL_PICK': 2.0,
  'MIN_PRED': 3225.07,
  'PER': 2507.16,
  'INJURY_PY': 0.0,
  'INFLATION': '5,523,948.00',
  'SAL_PRED': '9,864,132.00',
  'SAL_RISIDUAL': 1.79},
 {'POSITION': 'C',
  'PLAYER_NAME': 'Karl-Anthony Towns',
  'PHOTO': 'https://s3-us-west-2.amazonaws.com/static.fantasydata.com/headshots/nba/low-res/20001431.png',
  'YEAR': 2018.0,
  'YIL': 3.0,
  'ROUND_NUMBER': 1